### Implementing ReAct Agent with Native Function Calling
ReAct agent is an AI agent that follows the Reasoning and Acting (ReAct) framework. Modern LLMs increasingly offer native function calling capabilities, which streamline the process of integrating external tools into an agent's workflow. This approach allows the LLM to declare its intent to call a specific function and provide the necessary arguments in a structured format, typically JSON.

#### When to Use Native Function Calling

Native function calling is generally preferred when:

- The chosen LLM explicitly supports this feature (e.g., OpenAI's GPT models, Anthropic's Claude models, Google's Gemini models).
- Development speed and reduced parsing complexity are priorities. The LLM handles the formatting of the function call, minimizing the need for custom parsing logic.
- The interactions involve complex argument structures that are more easily represented in JSON than in plain text.



In [1]:
import openai
import json
import inspect
import re
from tools import web_search, calculator, weather_search, get_current_time
import os
from dotenv import load_dotenv
from openai import AzureOpenAI

SYSTEM_PROMPT_TEMPLATE = """
You are an expert assistant designed to solve complex tasks by reasoning step-by-step and interacting with available tools.
When handling time-related tasks or questions:
- If the user explicitly mentions a date, use that date as the reference point
- If no date is specified, try to get the current date from your tools
"""

In [2]:
def generate_tool_description(func):
  """
  Generate a tool definition from a function's signature and docstring.

  Args:
      func: The function to convert to a tool definition

  Returns:
      A dictionary containing the tool definition for LLM function calling
  """
  # Get function name and docstring
  func_name = func.__name__
  func_doc = inspect.getdoc(func) or ""

  # Get function description (first line of docstring)
  description = func_doc.split(
      '\n')[0] if func_doc else f"Call the {func_name} function"

  # Get function signature
  signature = inspect.signature(func)
  parameters = signature.parameters

  # Extract parameter descriptions from docstring using regex
  param_desc_pattern = re.compile(
      r'\s*Args:\s*\n(.*?)(?:\n\s*Returns:|$)', re.DOTALL)
  param_match = param_desc_pattern.search(func_doc)

  param_descriptions = {}
  if param_match:
    param_text = param_match.group(1)
    # Look for parameter descriptions in format: param_name: description
    param_pattern = re.compile(r'\s*(\w+):\s*(.+?)(?=\n\s*\w+:|$)', re.DOTALL)
    for match in param_pattern.finditer(param_text):
      param_name = match.group(1)
      description = match.group(2).strip()
      param_descriptions[param_name] = description

  # Build parameters object
  properties = {}
  required = []

  for param_name, param in parameters.items():
    param_type = "string"  # Default to string for simplicity
    description = param_descriptions.get(
        param_name, f"The {param_name} parameter")

    properties[param_name] = {
        "type": param_type,
        "description": description
    }

    # If parameter has no default value, mark it as required
    if param.default is inspect.Parameter.empty and param_name != 'self':
      required.append(param_name)

  # Create the tool definition
  tool = {
      "type": "function",
      "function": {
          "name": func_name,
          "description": description,
          "parameters": {
              "type": "object",
              "properties": properties,
              "required": required
          }
      }
  }

  return tool

# Function to generate the available tools description from the available_tools_map


def generate_tools_description(tools_map):
  """Generate a formatted description of available tools from the tools map."""
  return [generate_tool_description(tool_function) for tool_name, tool_function in tools_map.items()]


tools_map = {
    "web_search": web_search,
    "calculator": calculator,
    "weather_search": weather_search,
    "get_current_time": get_current_time,
}


def run_react_agent(user_query: str, llm_client, tools_map: dict, max_steps: int = 5):
  tools_description = generate_tools_description(tools_map)
  messages = []
  messages.append({
      "role": "system",
      "content": SYSTEM_PROMPT_TEMPLATE
  })
  messages.append({
      "role": "user",
      "content": user_query
  })
  print(f"User: {user_query}\n")
  for step in range(max_steps):
    print(f"*** Step {step + 1} of {max_steps}: ***\n")
    try:
      response = llm_client.chat.completions.create(
          model=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
          messages=messages,
          tools=tools_description,
          tool_choice="auto",
      )
    except openai.APIError as e:
      print(f"OpenAI API Error: {e}")
      messages.append(
          {"role": "assistant", "content": f"An API error occurred: {e}. Please try again."})
      break

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls

    if tool_calls:
      messages.append(response_message)  # Add assistant's turn with tool_calls
      for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)

        print(f"LLM wants to call: {function_name} with args: {function_args}")

        if function_name in tools_map:
          function_to_call = tools_map[function_name]
          try:
            function_response = function_to_call(**function_args)
            print(f"Observation (tool output): {function_response}\n")
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )
          except Exception as e:
            print(f"Error executing function {function_name}: {e}\n")
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": f"Error: {str(e)}",
                }
            )
        else:
          print(f"Error: Unknown function {function_name}\n")
          messages.append(
              {
                  "tool_call_id": tool_call.id,
                  "role": "tool",
                  "name": function_name,
                  "content": f"Error: Function {function_name} not found.",
              }
          )
    else:
      # No tool call, LLM provides a direct answer
      assistant_response = response_message.content
      print(f"Assistant: {assistant_response}\n")
      messages.append({"role": "assistant", "content": assistant_response})
      if assistant_response:  # Consider it final if there's content and no tool call
        return assistant_response  # Task considered complete

    if step == max_steps - 1:
      print("Max steps reached.")
      # Return the last assistant message or a summary
      final_content = messages[-1].get("content",
                                       "Could not resolve within max steps.")
      # If last message was a tool response, try to get a summary
      if messages[-1]["role"] == "tool":
        try:
          summary_response = llm_client.chat.completions.create(
              model=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
              messages=messages +
              [{"role": "user", "content": "Please summarize the current situation and provide a final answer if possible."}],
          )
          final_content = summary_response.choices.message.content
          print(f"Assistant (summary): {final_content}\n")
        except openai.APIError as e:
          print(f"OpenAI API Error during summary: {e}")
      return final_content

  # Fallback if loop finishes without a clear final answer from LLM without tool_calls
  last_assistant_message = next((m["content"] for m in reversed(
      messages) if m["role"] == "assistant" and m.get("content")), None)
  return last_assistant_message if last_assistant_message else "Agent could not determine a final answer."


In [3]:
load_dotenv()

# Set up Azure OpenAI client
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
)

# Update the call to run_custom_react_agent to include available_tools_map
final_answer = run_react_agent(
    "Where was the Microsoft founder was born? And how old is he/she now?",
    client,
    tools_map,
    max_steps=10)
print(f"Final Answer: {final_answer}")


User: Where was the Microsoft founder was born? And how old is he/she now?

*** Step 1 of 10: ***

LLM wants to call: web_search with args: {'query': 'Microsoft founder birthplace and birthdate'}
LLM wants to call: web_search with args: {'query': 'Microsoft founder birthplace and birthdate'}
Observation (tool output): Search failed with status code 202

*** Step 2 of 10: ***

Observation (tool output): Search failed with status code 202

*** Step 2 of 10: ***

LLM wants to call: web_search with args: {'query': 'Bill Gates place of birth and age'}
LLM wants to call: web_search with args: {'query': 'Bill Gates place of birth and age'}
Observation (tool output): Search failed with status code 202

*** Step 3 of 10: ***

Observation (tool output): Search failed with status code 202

*** Step 3 of 10: ***

LLM wants to call: web_search with args: {'query': 'Where was Bill Gates born and how old is he'}
LLM wants to call: web_search with args: {'query': 'Where was Bill Gates born and how old